## 임상시험 데이터 확인
### 목적
임상시험 데이터 및 포맷, 스펙(스키마)를 확인하고 임상시험 데이터의 최종적 형태인 sas7bdat / xport를 다룬다.

### 준비
임상시험 데이터: cafe.naver.com/dmisimportant/104

### Libraries
sas7bdat / xport를 열기 위해 sas7bdat 사용

In [1]:
import os
import numpy as np
import pandas as pd
import secrets
from sas7bdat import SAS7BDAT as sas7bdat

### Variables
CRF 포맷, LAB 데이터 포맷, 데이터셋 파일 경로 설정

In [2]:
ornament="-"*10
hangul="가나다라마바사아자차카파타하"
ext=(".sas7bdat",".xport")
path_crf="C:/code/CUBEDEMO2017/CUBEDEMO_2017_dc_or_fmt.xlsx"
path_lab="C:/code/CUBEDEMO2017/CUBEDEMO_2017_dc_or_lar.xlsx"
path_set="C:/code/CUBEDEMO2017/SASSET/"

### 포맷 파일 로드
CRF 포맷 파일에서 데이터 도메인 확인

In [3]:
form_crf=pd.read_excel(path_crf)
form_lab=pd.read_excel(path_lab)
print(ornament,"domain formats:",os.linesep,form_crf.DOMAIN.unique())

---------- domain formats: 
 ['AE' 'AY' 'CM' 'CT' 'CY' 'DM' 'DS' 'EF' 'EG' 'ES' 'IE' 'IP' 'LB' 'LC'
 'LY' 'MH' 'MY' 'PD' 'PG' 'RN' 'SU' 'SV' 'VS']


### 데이터셋 확인
데이터셋 경로의 sas7bdat / xport 파일을 os.path 오브젝트로 로드하고 불량 파일이 있는지 확인

In [4]:
sasobj=[obj for obj in os.scandir(path_set) if any(map(obj.path.lower().__contains__,ext)) and obj.is_file()]
sasbad=[obj for obj in sasobj if obj.stat().st_size<3]
if len(sasbad)>1:raise Exception("exotic file exists")

sas7bdat / xport os.path 오브젝트를 도메인 이름: 데이터프레임의 딕셔너리로 로드하고 어떤 도메인의 데이터셋이 있는지 확인
- 데이터셋 내 string이 bytes이므로 utf-8로 변환하고, 이 과정에서 누락되는 레코드가 있는지도 확인

In [5]:
def _decode(filepath):
    data=pd.read_sas(filepath)
    nas=data.notna().value_counts().sum()
    bytecol=data.select_dtypes("object").columns
    data[bytecol]=data[bytecol].apply(lambda q:q.str.decode("utf-8"))
    if nas==data.notna().value_counts().sum():
        return data
    else:
        print(ornament,"error:",filepath)
        return None

data={os.path.splitext(obj.name)[0].upper():_decode(obj.path) for obj in sasobj}
print(ornament,"domain:",os.linesep,data.keys(),os.linesep,len(data),"domains")

---------- domain: 
 dict_keys(['AE', 'AY', 'CM', 'CT', 'CY', 'DA', 'DM', 'DS', 'DY', 'EF', 'EG', 'EN', 'ES', 'IE', 'IP', 'LB', 'LC', 'LY', 'MH', 'MY', 'PD', 'PG', 'RN', 'SN', 'SU', 'SV', 'VS']) 
 27 domains


### 도메인별 데이터셋 형태 확인
SN은 subject 이름, visit site를 포함함을 알 수 있음

In [6]:
sn=data["SN"]
sn.sample(10)

,SUBJID,VISIT,SNNAME,SNDTC
14,S-2Z-035,5007.0,공우석,2017-02-20
11,S-2Z-029,5007.0,공우석,2017-02-20
21,S-MJ-009,5007.0,김민지,2017-06-14
20,S-MJ-002,5007.0,김민지,2017-06-20
26,S-US-007,5007.0,이상엽,2017-03-27
6,S-2Z-017,3006.0,황태중,2017-02-21
7,S-2Z-017,5007.0,황태중,2017-02-21
22,S-MJ-010,3006.0,김민지,2017-06-14
16,S-3Z-048,5007.0,송나경,2017-06-29
3,S-2Z-012,5007.0,신서란,2017-02-20


#### SN: SNNAME 가리기
Subject 이름을 가림

In [7]:
def _aname(name,n=2,chars=hangul):
    suffix="".join([secrets.choice(chars) for q in range(n)])
    return name[:n]+suffix

sn_snname_mapper={name:_aname(name) for name in sn.SNNAME.unique()}
sn.SNNAME.replace(sn_snname_mapper,inplace=True)
data["SN"].sample(10)

,SUBJID,VISIT,SNNAME,SNDTC
12,S-2Z-032,5007.0,송나하카,2017-02-20
15,S-3Z-013,3006.0,김민나하,2017-03-02
18,S-4Z-001,5007.0,원유다하,2017-03-07
14,S-2Z-035,5007.0,공우바라,2017-02-20
13,S-2Z-034,3006.0,송나하카,2017-02-20
1,S-1Z-034,5007.0,송민카하,2017-02-20
6,S-2Z-017,3006.0,황태카바,2017-02-21
17,S-4Z-001,3006.0,원유다하,2017-03-07
10,S-2Z-029,3006.0,공우바라,2017-02-20
25,S-US-003,5007.0,이상타사,2017-03-23


#### SN: SUBJID 추출
SN의 SUBJID를 각 도메인의 인덱스로 씀
- SN.SUBJID 개수를 subject 수로 가정함

In [8]:
ix=sn.SUBJID.unique()
print(ix)
print(ornament,"total subjects:",len(ix))

['S-1Z-031' 'S-1Z-034' 'S-2Z-006' 'S-2Z-012' 'S-2Z-013' 'S-2Z-016'
 'S-2Z-017' 'S-2Z-023' 'S-2Z-028' 'S-2Z-029' 'S-2Z-032' 'S-2Z-034'
 'S-2Z-035' 'S-3Z-013' 'S-3Z-048' 'S-4Z-001' 'S-MJ-001' 'S-MJ-002'
 'S-MJ-009' 'S-MJ-010' 'S-US-002' 'S-US-003' 'S-US-007']
---------- total subjects: 23


#### DM: 내용 확인
DM은 인구학적 정보를 포함하며, 왜인지 인덱스(SN.SUBJID)에 부합하지 않는 row가 많음
- 인구학적 정보는 시험 초기에 수집되므로 인덱스에 해당하지 않는 row가 있으면 안됨

In [9]:
dm=data["DM"]
dm
dm[dm.SUBJID.isin(ix)]

,SUBJID,BRTHDTC,AGE,SEX,FERTILE
23,S-1Z-031,1984-09,32.0,2.0,1.0
26,S-1Z-034,1977-11,38.0,2.0,1.0
37,S-2Z-006,1975-04,41.0,2.0,2.0
43,S-2Z-012,1977-10,39.0,2.0,1.0
44,S-2Z-013,1991-07,25.0,1.0,NaN
47,S-2Z-016,1987-02,29.0,1.0,NaN
48,S-2Z-017,1983-03,33.0,1.0,NaN
54,S-2Z-023,1998-04,17.0,1.0,NaN
58,S-2Z-028,1989-04,27.0,2.0,2.0
59,S-2Z-029,1975-04,41.0,1.0,NaN


#### AE: 내용 확인
AE는 시험간 subject 별 adverse effect 정보를 포함
- 컬럼(e.g. form)별 숫자 코딩된 categorical data가 있음
- 상기에 따라 MedDRA 코딩된 컬럼이 있음
- 숫자 코딩의 기준은 CRF Format에 정의되어 있음

In [10]:
ae=data["AE"]
print(ae.loc[:,[col for col in ae if col.startswith("AE")]].sample(10))
print(ae[ae.SUBJID.isin(ix)][["SUBJID","INV_PT"]].sample(10))
form_crf[form_crf.DOMAIN=="AE"].sample(10)

              AETERM     AESTDTC  AETEAE  AEOUT     AEENDTC  AESER  AESEV  \
72      Broken nails  2017-02-06     1.0    4.0  2017-02-11    1.0    1.0   
31             Fever  2016-10-26     2.0    2.0         NaN    1.0    1.0   
156            Death  2017-03-UK     2.0    2.0  2017-02-UK    2.0    2.0   
74   Cough increased  2016-11-29     1.0    2.0         NaN    3.0    2.0   
179              NaN         NaN     NaN    NaN         NaN    2.0    NaN   
78             ASPED  2017-02-02     1.0    4.0  2017-02-03    1.0    2.0   
16     Skin swelling  2016-12-01     1.0    3.0  2016-12-03    1.0    1.0   
48            Queasy  2017-01-UK     2.0    3.0  2017-02-01    1.0    1.0   
136     Tender sinus  2016-08-16     NaN    NaN         NaN    NaN    NaN   
52          AETEST01  2016-12-20     2.0    4.0  2017-02-UK    1.0    2.0   

     AEREL  AEACN  AEACNOTH  
72     2.0    6.0       3.0  
31     1.0    2.0       2.0  
156    2.0    2.0       2.0  
74     4.0    4.0       3.0  
17

,DOMAIN,FMTNAME,VARNAME,START,END,LABEL
0,AE,AEACN,AEACN,1,1,Dose increased
22,AE,AESER,AESER,2,2,Death
11,AE,AEOUT,AEOUT,1,1,Fatal
5,AE,AEACN,AEACN,6,6,Not applicable
4,AE,AEACN,AEACN,5,5,Drug withdrawn
13,AE,AEOUT,AEOUT,3,3,Recovered/Resolved
9,AE,AEACNOTH,AEACNOTH,3,3,Non-drug treatment
31,AE,AETEAE,AETEAE,1,1,Yes
20,AE,AEREL,AEREL,4,4,Related
7,AE,AEACNOTH,AEACNOTH,1,1,None


#### AE: AE Code Labeling
AE의 코딩된 categorical AEs이 CRF 상에서 무엇을 의미하는지 확인하고자 함
- DB Specification 또는 CRF Format에서 컬럼별 코드 및 라벨을 컬럼: 코드: 라벨의 딕셔너리로 추출
- 딕셔너리에 따라 컬럼 코드를 라벨링
- 일견 AESER rate가 상당히 높음

In [11]:
def _get_label(form,var="VARNAME",kvp=["END","LABEL"]):
    label={}
    for varname in form[var].unique():
        kv=form[form[var]==varname].loc[:,kvp].values
        label[varname]={k:v for k,v in kv}
    return label

ae_label=_get_label(form_crf)
print(ae_label)

{'AEACN': {1: 'Dose increased', 2: 'Dose not changed', 3: 'Dose reduced', 4: 'Drug interrupted', 5: 'Drug withdrawn', 6: 'Not applicable', 7: 'Unknown'}, 'AEACNOTH': {1: 'None', 2: 'Drug treatment', 3: 'Non-drug treatment', 4: 'Drug and non-drug treatment'}, 'AEOUT': {1: 'Fatal', 2: 'Not recovered/Not resolved', 3: 'Recovered/Resolved', 4: 'Recovered/Resolved with sequelae', 5: 'Recovering/Resolving', 6: 'Unknown'}, 'AEREL': {1: 'Not related', 2: 'Unlikely related', 3: 'Possibly related', 4: 'Related'}, 'AESER': {1: 'No', 2: 'Death', 3: 'Hospitalization', 4: 'Life threatening', 5: 'Congenital anomaly or birth defect', 6: 'Significant disability', 7: 'Other medically important event'}, 'AESEV': {1: 'Mild', 2: 'Moderate', 3: 'Severe'}, 'AETEAE': {1: 'Yes', 2: 'No'}, 'AEYN': {1: 'Yes', 2: 'No'}, 'CMINDC': {1: 'Medical history', 2: 'Adverse event', 3: 'Hypertension', 4: 'Prophylaxis', 5: 'Supplement', 6: 'Other'}, 'CMONGO': {1: 'Checked'}, 'CTYN': {1: 'Yes', 2: 'No'}, 'CMYN': {1: 'Yes', 2:

In [12]:
ae_col_ambi=[col for col in ae if col in form_crf[form_crf.DOMAIN=="AE"].VARNAME.values]
ae[ae_col_ambi].replace(ae_label)

,AETEAE,AEOUT,AESER,AESEV,AEREL,AEACN,AEACNOTH
0,Yes,Not recovered/Not resolved,Life threatening,Severe,Unlikely related,Dose reduced,Non-drug treatment
1,Yes,Unknown,Death,Mild,Related,Drug interrupted,None
2,NaN,Recovering/Resolving,NaN,NaN,NaN,NaN,NaN
3,Yes,Fatal,Life threatening,Moderate,Unlikely related,Dose not changed,Drug treatment
4,Yes,Recovered/Resolved with sequelae,Hospitalization,Moderate,Possibly related,Dose reduced,Drug treatment
...,...,...,...,...,...,...,...
186,No,Unknown,No,Mild,Not related,Dose not changed,Drug treatment
187,Yes,Not recovered/Not resolved,Hospitalization,Mild,Unlikely related,Drug withdrawn,Drug treatment
188,Yes,Fatal,No,Mild,Unlikely related,Unknown,None
189,No,Not recovered/Not resolved,Other medically important event,Mild,Possibly related,Drug withdrawn,Drug and non-drug treatment


#### LB: 내용 확인
LB는 랩 테스트 결과를 포함
- VISIT, SEQ 등 랩 테스트 회차에 따른 와이드 포맷으로 되어 있으며, 인덱스를 정할 때 유의해야 함
- LB.LBTEST에 해당 랩 테스트 명목이 나와 있음
- LB.LBORRES에 랩 결과 수치가 나와 있음
- 일부 categorical data 포함 (e.g. Occult Blood)

In [13]:
lb=data["LB"]
lb.sample(10)

,SUBJID,VISIT,SEQ,LBTEST,LBORRES,LBNOR,LBCLSIG
1597,S-2Z-027,2.0,8.0,Aspartate Aminotransferase,17.000,1.0,NaN
1351,S-2Z-020,1.0,15.0,Specific Gravity[U],0.003,2.0,1.0
714,S-1Z-033,6.0,9.0,Alanine Aminotransferase,21.000,1.0,NaN
3384,S-MJ-012,2.0,12.0,Creatinine,0.800,1.0,NaN
1758,S-2Z-033,2.0,10.0,Bilirubin,1.700,1.0,NaN
561,S-1Z-031,1.0,19.0,Protein[U],NaN,2.0,1.0
2263,S-3Z-029,1.0,4.0,Platelets,200.000,1.0,NaN
3372,S-MJ-012,1.0,19.0,Protein[U],NaN,1.0,NaN
1332,S-2Z-019,1.0,13.0,"Glomerular Filtration Rate, Estimated [Automat...",38.420,NaN,NaN
3482,S-MJ-019,2.0,14.0,"Glomerular Filtration Rate, Estimated(Cockcrof...",44.560,NaN,NaN


In [14]:
lb_desc=lb.groupby(["LBTEST"])["LBORRES"].agg(["mean","std","min","max","sum"])
lb_desc

,mean,std,min,max,sum
LBTEST,,,,,
Alanine Aminotransferase,336.257126,1409.494132,0.300,9083.352,64225.111
Albumin,335.514705,1577.394172,1.000,9954.012,63747.794
Albumin [U],NaN,NaN,NaN,NaN,0.000
Aspartate Aminotransferase,325.924723,1415.747206,1.000,9930.498,62251.622
Bilirubin,215.007593,1165.893477,0.400,9890.035,40636.435
Creatinine,184.471609,1056.242446,0.377,9596.921,35418.549
Erythrocytes,245.251260,1130.807728,1.000,7803.056,47088.242
"Glomerular Filtration Rate, Estimated [Automatical",5003.627111,3658.624184,782.108,9718.770,45032.644
"Glomerular Filtration Rate, Estimated [Automatically calculated]",82.635866,40.426010,0.080,194.760,14791.820


#### LB: Mockup Data 만들기
Subject Index와 LB를 이용해서 목업 데이터를 만듦
- SN.SUBJID를 기준으로 피험자당 10번의 랩 테스트를 가정함
- 편의상 DM 데이터를 결합
- 연습용 데이터 특성상 생성된 데이터는 현실성이 없음

In [15]:
def _gen_mockup_value(desc,count):
    print(ornament,"generating values")
    return {q:np.random.normal(desc.loc[q]["mean"],desc.loc[q]["std"],count) for q in desc.index}

def _gen_mockup(desc,ix,count=10):
    data=_gen_mockup_value(desc,count=len(ix)*count)
    ix=pd.MultiIndex.from_product([ix,[q for q in range(1,count+1,1)]],names=["SUBJID","VISIT"])
    return pd.DataFrame(data,ix).reset_index()

_gen_mockup(lb_desc.dropna(),ix).merge(dm,on="SUBJID")

---------- generating values


,SUBJID,VISIT,Alanine Aminotransferase,Albumin,Aspartate Aminotransferase,Bilirubin,Creatinine,Erythrocytes,"Glomerular Filtration Rate, Estimated [Automatical","Glomerular Filtration Rate, Estimated [Automatically calculated]",...,Hemoglobin A1C,Leukocytes,Platelets,Protein,Specific Gravity[U],pH[U],BRTHDTC,AGE,SEX,FERTILE
0,S-1Z-031,1,-574.410719,-2716.259401,-1818.460516,379.679147,263.827859,459.248496,-2141.228205,89.264975,...,-246.377095,1419.170484,948.723899,351.354867,-860.733830,2763.544648,1984-09,32.0,2.0,1.0
1,S-1Z-031,2,1128.817638,-301.070869,-180.188763,-234.989789,738.578726,16.711296,1659.212923,70.757657,...,1478.498564,2596.948928,-353.474718,1249.687657,-1028.475767,1261.521332,1984-09,32.0,2.0,1.0
2,S-1Z-031,3,209.341560,-672.506189,2444.053645,-719.830450,1432.182224,2491.101921,6456.701728,108.945183,...,-603.774572,1447.082608,1241.384393,929.016069,1299.216366,1429.384566,1984-09,32.0,2.0,1.0
3,S-1Z-031,4,-914.099585,243.401863,-331.339711,849.518185,-1072.032715,-1368.745683,7004.542585,51.652873,...,238.280661,2325.020159,2269.860308,-1808.947327,86.652155,-965.398062,1984-09,32.0,2.0,1.0
4,S-1Z-031,5,-1148.376835,-1242.092250,1462.008455,-354.276589,-520.843866,-587.212481,8112.062734,46.495220,...,892.721695,589.566891,715.166192,-195.508658,470.594092,2145.521394,1984-09,32.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,S-US-007,6,-23.454735,401.680067,-1720.553915,263.168136,336.475758,-1559.691917,2852.680001,46.544348,...,-1070.829118,708.765697,1079.375845,729.542056,201.731123,128.088904,1981-01,36.0,1.0,NaN
226,S-US-007,7,-696.490307,2353.479925,-3077.738532,1565.221125,614.917834,1491.303219,6964.424682,49.463241,...,1526.089725,-448.679501,976.505903,-30.713363,-8.121181,-382.068627,1981-01,36.0,1.0,NaN
227,S-US-007,8,1311.224018,1776.501295,-1486.072987,-369.086641,1565.252958,2553.884682,6978.578084,30.870955,...,2013.914177,2055.079059,-67.995950,985.118123,-76.541200,-640.733639,1981-01,36.0,1.0,NaN
228,S-US-007,9,1400.006365,1231.857566,1195.046753,1153.611956,152.833624,584.212650,4869.563282,84.255562,...,1791.273668,1356.710027,-876.787473,3661.837521,-961.737844,-604.539617,1981-01,36.0,1.0,NaN


### 후기
- 하나의 임상시험을 구성하는 데이터셋은 많으며, 각 데이터 연동을 위한 정의와 연동이 일목요연히 되어 있음
    - 수치형, 범주형 값이 혼재함
    - 일반적이지 않은 범주형 값 (0,1 No,Yes(False,True)가 아닌 1,2 Yes,No)
    - 데이터셋 복잡도의 중요한 요인은 프로토콜의 site visit number로 생각됨
- 분석에 이용되는 실질적 데이터뿐만 아니라 엔드유저에게 해당 데이터가 어떻게 보이고 수집됐는지를 포함
- 프로토콜 이해를 선행하지 않고서는 정확히 데이터셋을 다루기 어려움
    - Visit Number / Window, Endpoint, Inclusion / Exclusion Criteria
    - Longitudinal Study 여부 (endpoint가 time series인지)
- 도메인 데이터와 DB Specification 내 각 도메인 정의를 연동할 수 있는 클래스 이용 필요